In [40]:
import csv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import scipy as sp
import pandas as pd
import pycountry_convert as pc
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from merge_datasets import MergeDatasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

In [29]:
co2_df = pd.read_csv('../datasets/co2-long-term-concentration.csv')
forest_df = pd.read_csv('../datasets/forest-area-km.csv')
land_df = pd.read_csv('../datasets/land-use-over-the-long-term.csv')
temp_df = pd.read_csv('../datasets/annual-temperature-anomalies.csv')
invas_df = pd.read_csv('../datasets/budget-to-manage-invasive-alien-species.csv')
lpi_df = pd.read_csv('../datasets/global-living-planet-index.csv') # target

merge_datasets = MergeDatasets(co2_df, forest_df, land_df, temp_df, invas_df, lpi_df)
merged_df = merge_datasets.merge()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(merged_df)

                              Entity      Code  Year  Living Planet Index  \
0                             Africa       NaN  1990            53.534120   
1                   Asia and Pacific       NaN  1990            90.093490   
2            Europe and Central Asia       NaN  1990           130.544690   
3    Latin America and the Caribbean       NaN  1990            34.995760   
4                      North America       NaN  1990            85.231450   
5                              World  OWID_WRL  1990            62.681120   
6                             Africa       NaN  1991            52.495760   
7                   Asia and Pacific       NaN  1991            86.526734   
8            Europe and Central Asia       NaN  1991           130.741070   
9    Latin America and the Caribbean       NaN  1991            33.752610   
10                     North America       NaN  1991            84.330565   
11                             World  OWID_WRL  1991            61.609140   

In [3]:
class LocallyWeightedLR():
    
    def __init__(self, X, y, tau):
        self.X = X
        self.y = y
        self.tau = tau 
        
    def compute_weights(self, x):
        m = self.X.shape[0]
        W = np.eye(m)
        # x = np.hstack((1, x))
        
        for i in range(m):
            diff = self.X[i] - x
            W[i, i] = np.exp(-diff.T.dot(diff) / (2 * self.tau ** 2))
            
        return W
    
    def compute_theta(self, x):
        W = self.compute_weights(x)
        lambda_identity = np.identity(self.X.shape[1]) * 0.001  # small regularization term
        XTWX = self.X.T.dot(W).dot(self.X) + lambda_identity  # add regularization
        theta = np.linalg.inv(XTWX).dot(self.X.T).dot(W).dot(self.y)
        return theta
    
    # prediction for an input x
    # also return the local linear regression parameters (theta) for this x.
    def predict(self, x):
        theta = self.compute_theta(x)
        # x = np.hstack((1, x))
        prediction = x.dot(theta)
        return prediction, theta

In [66]:
features = ['Year', 'Entity', 'Long-run CO₂ concentration', 'Forest area', 'Land use: Built-up area', 
            'Land use: Grazingland', 'Land use: Cropland', 'Temperature anomaly']

entities = pd.get_dummies(merged_df['Entity'])
X_continuous = merged_df[['Year', 'Long-run CO₂ concentration', 'Forest area', 'Land use: Built-up area', 
                          'Land use: Grazingland', 'Land use: Cropland', 'Temperature anomaly']].values
X = np.hstack((X_continuous, entities.values))

y = merged_df['Living Planet Index'].values.reshape(-1, 1)
X_design_mat = np.hstack((np.ones((X.shape[0], 1)), X))

lwr_model = LocallyWeightedLR(X_design_mat, y, tau=0.5)

In [68]:
predictions = []
for i in range(20):
    pred, _ = lwr_model.predict(X_design_mat[i, :])
    predictions.append(pred)

predictions = np.array(predictions).flatten()
actuals = y[:20].flatten()

mse = mean_squared_error(actuals, predictions)
print(f"Mean Squared Error: {mse}")

for i in range(len(predictions)):
    print(f"Actual LPI: {actuals[i]}, Predicted LPI: {predictions[i]}")

Mean Squared Error: 3710411.8418504163
Actual LPI: 53.53412, Predicted LPI: 73.81837761237182
Actual LPI: 90.09349, Predicted LPI: 5641.695742823218
Actual LPI: 130.54469, Predicted LPI: 793.0961550092369
Actual LPI: 34.99576, Predicted LPI: -103.99567676669534
Actual LPI: 85.23145, Predicted LPI: 148.77471365961128
Actual LPI: 62.68112, Predicted LPI: -527.1745275191579
Actual LPI: 52.49576, Predicted LPI: 802.2932222495709
Actual LPI: 86.526734, Predicted LPI: 107.10413140655224
Actual LPI: 130.74106999999998, Predicted LPI: 503.52776477030295
Actual LPI: 33.75261, Predicted LPI: 25.11992605672768
Actual LPI: 84.330565, Predicted LPI: -486.3731773477253
Actual LPI: 61.60914, Predicted LPI: -155.51691359058273
Actual LPI: 52.553064000000006, Predicted LPI: 502.71853439562585
Actual LPI: 82.85733499999999, Predicted LPI: -25.883764973960805
Actual LPI: 128.74790000000002, Predicted LPI: -4442.168480271268
Actual LPI: 32.369485, Predicted LPI: 0.7791289484158476
Actual LPI: 84.54887, Pr

Initial testing of locally weighted regression shows really poor performance. We will attempt regularization techniques and other techniques to see if we can improve the performance.

In [61]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

elastic_net_model = ElasticNet(alpha=0.01, l1_ratio=0.5)
elastic_net_model.fit(X_train, y_train)

elastic_predictions = elastic_net_model.predict(X_test)

elastic_mse = mean_squared_error(y_test, elastic_predictions)
print(f"ElasticNet Regression MSE: {elastic_mse}")

ElasticNet Regression MSE: 8.720568434190678
